## 주제
#### Random Forest를 이용한 대출 상환 여부 예측
## 주제 선택 GPT 프롬프트 링크
https://chatgpt.com/share/674d7cce-4774-800a-acad-d3315034eafd

## 사용 라이브러리

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
RANDOM_STATE = 45

## CSV 데이터 로딩 및 가공

In [2]:
file_csv = pd.read_csv("train.csv")

In [3]:
file_csv.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
#NaN값 확인
file_csv.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [5]:
#NaN값 제거
file_csv = file_csv.dropna()
file_csv.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [6]:
#NaN값 제거 후 데이터 확인
file_csv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 480 entries, 1 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            480 non-null    object 
 1   Gender             480 non-null    object 
 2   Married            480 non-null    object 
 3   Dependents         480 non-null    object 
 4   Education          480 non-null    object 
 5   Self_Employed      480 non-null    object 
 6   ApplicantIncome    480 non-null    int64  
 7   CoapplicantIncome  480 non-null    float64
 8   LoanAmount         480 non-null    float64
 9   Loan_Amount_Term   480 non-null    float64
 10  Credit_History     480 non-null    float64
 11  Property_Area      480 non-null    object 
 12  Loan_Status        480 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 52.5+ KB


## One-hot encoding

In [7]:
#이진 변수를 제외한 범주형 변수 계산
train_variables = ['Gender',
                   'Married',
                   'Dependents',
                   'Education',
                   'Self_Employed',
                   'Property_Area'
                  ]

In [8]:
file_csv = pd.get_dummies(data = file_csv,
                            prefix = train_variables,
                            columns = train_variables
                           )

In [9]:
#필요없는 데이터 열 삭제 및 변수 세트 설정
file_csv = file_csv.drop(['Loan_ID'], axis = 1)
X = file_csv.drop(['Loan_Status'], axis = 1)
Y = file_csv['Loan_Status']

file_csv.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Female,Gender_Male,Married_No,Married_Yes,...,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
1,4583,1508.0,128.0,360.0,1.0,N,False,True,False,True,...,True,False,False,True,False,True,False,True,False,False
2,3000,0.0,66.0,360.0,1.0,Y,False,True,False,True,...,False,False,False,True,False,False,True,False,False,True
3,2583,2358.0,120.0,360.0,1.0,Y,False,True,False,True,...,False,False,False,False,True,True,False,False,False,True
4,6000,0.0,141.0,360.0,1.0,Y,False,True,True,False,...,False,False,False,True,False,True,False,False,False,True
5,5417,4196.0,267.0,360.0,1.0,Y,False,True,False,True,...,False,True,False,True,False,False,True,False,False,True


## RandomForest / GridSearchCV

### GridSearchCV란?
머신러닝에서 하이퍼파라미터의 최적값을 찾기위해 사용되는 기법 중 하나이다.

사용자가 지정한 파라미터에 대해 모든 경우의 수를 통해 성능이 가장 좋은 파라미터 최적값을 찾아주는 함수가 존재한다.



class sklearn.model_selection.GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)

In [10]:
#Train : test = 7.5 : 2.5 비율로 split
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size = 0.75, random_state = RANDOM_STATE)

In [11]:
#GridSearchCV를 사용하기 위해 RandomForest 모델을 먼저 제작
model = RandomForestClassifier(random_state = RANDOM_STATE, n_jobs = -1) #n_jobs = -1 -> 병렬 계산을 위해 CPU 개수를 설정 -1이면 모든 CPU 사용
params = { 'max_depth' : [2, 4, 8, 16, 32, 64, None],
           'min_samples_leaf' : [2, 4, 6, 8],
           'min_samples_split' : [2, 5, 10, 15, 30, 40, 50, 75, 100],
           'n_estimators' : [10, 25, 50, 75, 100, 200],
         }

In [12]:
#RandomForest 최적의 파라미터를 위해 GridSearchCV 사용
grid_cv = GridSearchCV(model, param_grid = params, n_jobs = -1)

In [13]:
#훈련세트 학습하기
grid_cv.fit(x_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(n_jobs=-1, random_state=45),
             n_jobs=-1,
             param_grid={'max_depth': [2, 4, 8, 16, 32, 64, None],
                         'min_samples_leaf': [2, 4, 6, 8],
                         'min_samples_split': [2, 5, 10, 15, 30, 40, 50, 75,
                                               100],
                         'n_estimators': [10, 25, 50, 75, 100, 200]})

In [14]:
print('최적 하이퍼 파라미터: ', grid_cv.best_params_)

최적 하이퍼 파라미터:  {'max_depth': 8, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}


In [15]:
#최적 하이퍼 파라미터를 기반으로 최종모델 제작
final_model = RandomForestClassifier(max_depth = 8,
                                     min_samples_leaf = 2,
                                     min_samples_split = 10,
                                     n_estimators = 50,
                                     random_state = RANDOM_STATE
                                    )

In [16]:
#최종 모델을 기반으로 훈련세트 학습하기
final_model.fit(x_train, y_train)

RandomForestClassifier(max_depth=8, min_samples_leaf=2, min_samples_split=10,
                       n_estimators=50, random_state=45)

# 결과

In [17]:
print('훈련 세트 예측 정확도 : {:.3f}'.format(accuracy_score(final_model.predict(x_train), y_train)))

훈련 세트 예측 정확도 : 0.839


In [18]:
print('테스트 세트 예측 정확도 : {:.3f}'.format(accuracy_score(final_model.predict(x_test), y_test)))

테스트 세트 예측 정확도 : 0.833


## GPT 사용
### 결측값 처리에 대한 방법

GPT URL : https://chatgpt.com/share/674db238-2ef0-800a-8492-34df93fe776a

## 첫 번째 질문 프롬프트
#### **데이터셋에 null값이 너무 많아 이 null값이 처리하는 방법을 알려줘**

### 1. null 값 제거
    행 삭제: null 값이 많은 행을 삭제합니다.
    df.dropna()를 사용하여 null 값이 있는 행 전체를 삭제할 수 있습니다.
    단, 삭제로 인해 데이터 손실이 크지 않아야 효과적입니다.
    열 삭제: null 값 비율이 높은 열을 삭제합니다.
    예: null 값이 50% 이상인 열을 삭제.

    df = df.dropna(thresh=int(len(df) * 0.5), axis=1)

### 2. null 값 대체 (Imputation)
    평균, 중앙값 또는 최빈값으로 대체
    수치형 데이터: mean(), median()으로 대체.
    범주형 데이터: mode()로 대체.

    df['column'] = df['column'].fillna(df['column'].mean())
    고급 대체 (예측값으로 대체)
    회귀, KNN, 머신러닝 모델을 사용해 null 값을 예측하여 채웁니다.

    from sklearn.impute import KNNImputer
    imputer = KNNImputer(n_neighbors=3)
    df_imputed = imputer.fit_transform(df)

### 3. 특별한 값으로 대체
    null 값 자체를 의미 있는 값으로 변환.
    예: 'Unknown', 0, -1 등의 값으로 대체.

    df['column'] = df['column'].fillna('Unknown')

### 4. 특정 패턴이나 그룹을 고려한 대체
    그룹별 통계치를 사용해 null 값을 대체.
    예: 각 카테고리별 평균값으로 대체.

    df['column'] = df.groupby('category')['column'].transform(lambda x: x.fillna(x.mean()))

### 5. null 값 여부를 별도 피처로 저장
    null 값이 있었는지 여부를 새로운 열에 기록.

    df['column_null'] = df['column'].isnull().astype(int)

### 6. 모델 특성을 활용
    머신러닝 모델의 경우, null 값을 자체적으로 처리할 수 있는 알고리즘을 사용.
    예: XGBoost, LightGBM 등은 null 값을 그대로 입력으로 사용할 수 있음.

## 두 번째 질문 프롬프트
#### **Null값이 들어있는 데이터들이 평균값 및 다른 값으로 대체하기가 어려워 그럴때에는 데이터 표본 수가 줄어들더라도 데이터 해당 행을 삭제하는게 좋을지 알려줘**

### 삭제를 고려해야 할 상황
##### 1. Null 값이 데이터셋의 작은 부분을 차지하는 경우

    Null 값이 있는 행이 데이터셋의 5~10% 미만이라면 삭제해도 분석에 큰 영향을 미치지 않을 가능성이 높습니다.
    df = df.dropna()

##### 2. 대체 불가능하거나 의미 있는 값을 추정하기 어려운 경우

    Null 값이 대체할 방법이 없고, 다른 데이터를 손상시킬 위험이 있다면 해당 데이터를 제거하는 것이 더 나을 수 있습니다.

##### 3. Null 값의 분포가 랜덤한 경우

    Null 값이 데이터의 특정 그룹에 치우치지 않고 무작위로 분포해 있으면, 삭제해도 데이터의 대표성을 크게 훼손하지 않을 가능성이 있습니다.

### 삭제를 피하거나 보완해야 할 상황
##### 1. 데이터셋이 작아서 삭제가 큰 영향을 미칠 경우

    데이터셋이 작아 한 행만 삭제해도 분석 결과에 큰 영향을 미치는 경우, 다른 방법(예: 특수한 값으로 대체)을 고려하세요.

##### 2. Null 값이 중요한 패턴을 나타낼 가능성이 있는 경우

    Null 값이 특정 그룹이나 상황에서 나타나는 중요한 특성일 가능성이 있다면, 이를 삭제하지 않고 분석에 활용해야 할 수 있습니다.

    df['column_null'] = df['column'].isnull().astype(int)  # Null 여부를 별도 열로 저장

### 데이터 삭제 전 대안
##### 1. 특정 열을 기준으로 그룹화 후 평균값 또는 중앙값으로 대체

    동일한 그룹의 값으로 대체하면 데이터 삭제를 줄일 수 있습니다.

    df['column'] = df.groupby('category')['column'].transform(lambda x: x.fillna(x.median()))

##### 2. 모델 기반 대체

    머신러닝 알고리즘으로 Null 값을 예측해 대체합니다.

    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(strategy='most_frequent')
    df['column'] = imputer.fit_transform(df[['column']])

##### 3. Null 값 비율에 따라 열 또는 행 선택적 삭제

    Null 비율이 높은 행이나 열만 삭제합니다.

    df = df.dropna(thresh=int(df.shape[1] * 0.8))  # 80% 이상 값이 있는 행만 남김

### 결론
    데이터셋 크기가 크고 Null 값의 비율이 낮다면 삭제를 추천합니다.
    데이터셋 크기가 작거나 Null 값이 많은 경우, 대체 가능성을 최대한 탐색한 후 삭제 여부를 결정하세요.

## 세 번째 질문 프롬프트
#### **데이터 개수는 613개, feature 개수는 12개 그중 null값이 포함된 데이터 행의 개수는 143개야. 무엇보다 null값을 가진 feature가 이진 변수인 것이 있어 예측하기가 힘들어. 이럴때에는 어떻게 해야할까?**


### 처리 전략
##### 1. Null 값을 대체하는 방법 (삭제를 최소화하는 방향)
    이진 변수는 0과 1처럼 제한된 값만 가질 수 있으므로 예측하기 어려운 특성이 있습니다. 이런 경우, 다음 대체 방법을 고려할 수 있습니다:

    a. 최빈값(Most Frequent Value)으로 대체
        이진 변수라면, 전체 데이터에서 0 또는 1 중 더 많이 나타나는 값으로 null 값을 대체합니다.

        from sklearn.impute import SimpleImputer

        imputer = SimpleImputer(strategy='most_frequent')
        df['binary_feature'] = imputer.fit_transform(df[['binary_feature']])

        장점: 간단하고 빠르며 데이터 손실 없음.
        단점: 대체된 값이 패턴에 맞지 않을 수 있음.

    b. 그룹 기반 대체
        다른 Feature를 활용해 그룹을 나누고, 각 그룹에서 이진 변수의 최빈값으로 대체합니다.

        df['binary_feature'] = df.groupby('another_feature')['binary_feature'].transform(
        lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 0))
        
        예: another_feature에 따라 분류된 그룹별로 이진 변수의 Null 값을 채움.

    c. 새로운 카테고리 추가
        Null 값을 별도의 값(e.g., -1)로 대체하여, 데이터 자체에서 Null이 있었음을 모델이 인식할 수 있게 합니다.

        df['binary_feature'] = df['binary_feature'].fillna(-1)
        
        장점: Null 값 자체를 분석에 반영.
        단점: 모델이 추가적인 정보를 잘 해석해야 함.

##### 2. Null 값이 있는 데이터 행 삭제
    Null 값이 포함된 데이터 행이 약 23%라면, 데이터셋 크기가 충분히 커 분석에 큰 영향을 미치지 않는다면 삭제할 수 있습니다.

    df = df.dropna()

    장점: 간단하며 Null 값이 분석에 영향을 미치지 않음.
    단점: 데이터 손실이 약 23% 발생.

##### 3. Null 값 여부를 새로운 Feature로 추가
    Null 값이 데이터의 특정 패턴을 나타낼 수 있으므로 Null 여부를 별도의 Feature로 저장하여 분석에 활용할 수 있습니다.
    
    df['binary_feature_null'] = df['binary_feature'].isnull().astype(int)
    Null 값을 채운 뒤, 새로 추가된 Feature를 모델링에 사용.

##### 4. Null 값 예측 및 대체 (머신러닝 활용)
    이진 변수에 Null 값이 많을 경우, 나머지 Feature를 사용해 Null 값을 예측하는 머신러닝 모델을 훈련시킬 수 있습니다.

    from sklearn.ensemble import RandomForestClassifier

    #Null이 아닌 데이터로 모델 학습
    train = df[df['binary_feature'].notnull()]
    test = df[df['binary_feature'].isnull()]

    model = RandomForestClassifier()
    model.fit(train.drop(columns='binary_feature'), train['binary_feature'])

    #Null 값 예측 후 대체
    predicted = model.predict(test.drop(columns='binary_feature'))
    df.loc[df['binary_feature'].isnull(), 'binary_feature'] = predicted

    장점: 데이터 특성을 반영해 Null 값을 대체.
    단점: 추가적인 계산 필요 및 모델 정확도 의존.

### 추천 접근 방법
    -이진 변수의 Null 값 대체-
    Null 값을 분석에 포함하려면 그룹 기반 대체나 새로운 카테고리 추가를 우선적으로 고려하세요.

    -전체 데이터셋 처리-
    Null 값을 예측하기 어렵고 분석에 중요하지 않은 Feature라면 Null 값이 포함된 행 삭제도 합리적인 선택입니다.
    데이터 손실을 최소화하려면, 머신러닝 기반 대체를 고려해보세요.